## 对数据增强

In [ ]:
import os
import random
import torch
#tensor是python中基本数据结构，类似于numpy的数组，支持并行计算
#nn构建神经网络，包括常用的层(卷积层，全连接层，激活函数)，损失函数和模块
from torch import nn,Tensor
#image 进行图像创建和绘制，图像转换和图像处理
#imageDraw 用于在图像上绘制图形，创建简单的图形
from PIL import Image,ImageDraw
# pytorch对图像进行常见的转换和处理，这些函数可以在图像预处理和数据增强中使用
from torchvision.transforms import functional as F
import numpy as np
import cv2  #opencv可以进行图像处理，视频处理，计算机视觉(人脸检测，图像分割)
#用于数据增强，生成更多样本提高模型的性能和鲁棒性
#支持多种数据类型，可以处理单张图像、批量图像以及图像和其对应标签进行组合
import imgaug.augmenters as iaa


In [ ]:
#组合转换类
class Compose(object):
        #transform 是变换数组
        def __init__(self, transforms):
                self.tranforms = transforms
        
        def __call__(self, image):
                for t in self.tranforms:
                        image = t(image)
                return image
            

In [ ]:
#图像转为张量
#主要目的是为了能够在深度学习框架中进行高校的计算和训练
class ToTensor(object):
        def __call__(self, image):
                images=torch.tensor(image,dtype=torch.float32) #将图片转换为向量
                #permute许多深度学习框架中在卷积神经网络要求输入数据使用(batch,channels,height,width)
                #contiguous()把非连续张量转化为连续的张量
                images.permute((0,3,1,2)).contiguous() 
                return images

In [ ]:
#对输入图像进行归一化处理，改善模型的训练效果和收敛速度，消除不同特征之间的偏差，帮助模型更好的学习，避免梯度消失(解决措施：使用relu激活函数，归一化)
#通道数就是特征数
class Normalize(object):
        def __init__(self, mean, std,inplace=False):
                self.mean=mean
                self.std=std
                self.inplace=inplace
        
        def __call__(self,images):
                for i in range(len(images)):
                        images[i]=F.normalize(images[i],self.mean,self.std,self.inplace) #使处理后的数据更接近标准正态分布
                return images

In [ ]:
#随机水平翻转，增强训练数据的多样性
class RandomHorizontalFlip(object):
        def __init__(self,prob=0.5):
                self.prob=prob
        
        def __call__(self,image):
                if random.random()<self.prob:
                        image=image.flip(-1) #在宽度方向进行反转
                return image
        

In [ ]:
#图片填充和调整大小，目的是将输入图像调整到指定大小进行填充，特别是在图像和目标检测中，以确保所有输入图像具有相同尺寸，便于批处理和模型训练



In [12]:
from uiautomation import WindowControl
from zhipuai import ZhipuAI

wx = WindowControl(Name='微信', searchDepth=1)
print(wx)
# 切换窗口
wx.SwitchToThisWindow()
# 寻找会话控件
session = wx.ListControl(Name='会话')
# 监听消息
while True:
    msg = session.TextControl(searchDepth=4)
    while not msg.Exists(0):
        pass
    if msg.Name:
        msg.Click(simulateMove=False)
        # 获取最后一条消息
        msg_content = wx.ListControl(Name='消息').GetChildren()[-1].Name
        print(msg_content)
        # 调用大模型 - 单次会话
        client = ZhipuAI(api_key="b8d2755fc76e7cc087bc3422eaa6ae69.i7Txc9rS66CCwI2q")
        response = client.chat.completions.create(
            model="glm-4",  # 请填写您要调用的模型名称
            messages=[
                {"role": "user", "content": msg_content}
            ],
        )
        print(response.choices[0].message)
        reply = response.choices[0].message.content
        # 替换换行符
        wx.SendKeys(reply.replace('{br}', '{Shift}{Enter}'), waitTime=0)
        wx.SendKeys('{Enter}', waitTime=0)
        # 复位
        session.Click()

[文件]
CompletionMessage(content='看起来您可能想要发送或讨论一个文件，但遗憾的是，作为一个文本交互的AI，我无法直接接收或查看文件。如果您有关于文件的问题或需要帮助，请用文字描述它，我会尽我所能提供帮助。', role='assistant', tool_calls=None)
[文件]
CompletionMessage(content='看起来您可能想要发送一个文件或者询问有关文件的问题，但遗憾的是，作为一个文本交互的AI，我无法直接接收或查看文件。如果您有关于文件的问题或者需要帮助，请用文字描述您的需求，我会尽力提供帮助。', role='assistant', tool_calls=None)
[文件]
CompletionMessage(content='看起来您可能想要分享一个文件或者询问有关文件的问题，但是没有文件被实际发送或附加。如果您有文件需要帮助，比如需要解读某个文件内容、需要帮助编辑或者有关于文件的其他问题，请详细描述您的需求，我会尽力提供帮助。如果需要上传文件，请确保它不包含敏感信息，并且符合相关政策和规定。由于我目前无法直接接收文件，您可以将文件内容以文本形式分享出来。', role='assistant', tool_calls=None)
[文件]
CompletionMessage(content='对不起，但作为一个文本和图片处理的AI，我无法接收或查看文件。如果您有具体的问题或需要信息，请直接在这里提问，我会尽我所能来帮助您。', role='assistant', tool_calls=None)
[文件]
CompletionMessage(content='看起来您可能想要分享一个文件，但作为一个文本交互式的AI，我无法接收或查看文件。如果您有关于文件的问题或需要帮助，请描述您的情况或询问，我会尽力提供帮助。', role='assistant', tool_calls=None)
[文件]
CompletionMessage(content='对不起，但作为一个文本和图像处理的AI，我无法直接接收或查看文件。如果你有具体的问题或者需要帮助的地方，欢迎你用文字描述，我会尽力提供帮助。如果需要处理文件内容，请将相关信息以文本形式提供。', role='assistant', tool_calls=

COMError: (-2146233083, None, (None, None, None, 0, None))